<a href="https://colab.research.google.com/github/adithya-prabhu-22/deep_learning_algos/blob/main/cat_dog_classificationj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================
# STEP 1 — Import Libraries
# =========================================
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf

# =========================================
# STEP 2 — Extract ZIP
# =========================================
zip_path = "/content/cat_dog_mix.zip"  # change if different
extract_path = "/content/cat_dog_mix"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped files to:", extract_path)

# =========================================
# STEP 3 — Load Images and Labels
# =========================================
image_dir = os.path.join(extract_path, "cat_dog_mix")

images = []
labels = []

files = sorted(os.listdir(image_dir), key=lambda x: int(x.split('.')[0]))
 # ensure numeric order

for idx, file in enumerate(files):
    if file.endswith(".jpg"):
        path = os.path.join(image_dir, file)
        img = cv2.imread(path)
        img = cv2.resize(img, (128, 128))  # resize to small fixed size
        images.append(img)

        # Label assignment
        label = 0 if idx < 1440 else 1  # 0=cat, 1=dog
        labels.append(label)

X = np.array(images, dtype='float32') / 255.0  # normalize to [0,1]
y = np.array(labels)

print("✅ Loaded images:", X.shape, "Labels:", y.shape)

# =========================================
# STEP 4 — Split Train & Validation
# =========================================
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train set:", X_train.shape, "Validation set:", X_val.shape)

# =========================================
# STEP 5 — Data Augmentation
# =========================================
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

# =========================================
# STEP 6 — CNN Model (with Dropout)
# =========================================
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# =========================================
# STEP 7 — Training (with Early Stopping)
# =========================================
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=15,
    callbacks=[callback],
    verbose=1
)

# =========================================
# STEP 8 — Plot Training Results
# =========================================
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

# =========================================
# STEP 9 — Evaluate
# =========================================
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"✅ Validation Accuracy: {val_acc*100:.2f}%")

# Save model
model.save("cat_dog_cnn.h5")
print("✅ Model saved as cat_dog_cnn.h5")


✅ Unzipped files to: /content/cat_dog_mix
✅ Loaded images: (2879, 128, 128, 3) Labels: (2879,)
Train set: (2303, 128, 128, 3) Validation set: (576, 128, 128, 3)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.4874 - loss: 0.8173 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.5033 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/15
13/72 ━━━━━━━━━━━━━━━━━━━━ 1:14 1s/step - accuracy: 0.4766 - loss: 0.6933